In [ ]:
# 1) YOLOv8 설치
!pip install ultralytics --upgrade


In [4]:

# 2) 라이브러리 및 모델 불러오기
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

# 3) 이미지 로드 (예: 인터넷에서 샘플 이미지를 다운로드)
!wget -O sample.jpg https://ultralytics.com/images/bus.jpg

# 4) 모델 불러오기 및 예측
model = YOLO('yolov8n.pt')  # 가장 가벼운 버전
results = model('sample.jpg')

# 5) 결과 시각화
for r in results:
    r.show()

--2025-07-18 15:10:15--  https://ultralytics.com/images/bus.jpg
Resolving ultralytics.com (ultralytics.com)... 75.2.70.75, 99.83.190.102
Connecting to ultralytics.com (ultralytics.com)|75.2.70.75|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.ultralytics.com/images/bus.jpg [following]
--2025-07-18 15:10:15--  https://www.ultralytics.com/images/bus.jpg
Resolving www.ultralytics.com (www.ultralytics.com)... 104.18.40.102, 172.64.147.154
Connecting to www.ultralytics.com (www.ultralytics.com)|104.18.40.102|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/ultralytics/assets/releases/download/v0.0.0/bus.jpg [following]
--2025-07-18 15:10:15--  https://github.com/ultralytics/assets/releases/download/v0.0.0/bus.jpg
Resolving github.com (github.com)... 20.200.245.247
Connecting to github.com (github.com)|20.200.245.247|:443... connected.
HTTP request sent, awaiting response... 3

RuntimeError: Numpy is not available

In [ ]:
# 여러 이미지 일괄 탐지 예제
import glob
import os
import requests # Import the requests library

# 1) 샘플 이미지 여러 개 다운로드
img_urls = [
    "https://ultralytics.com/images/bus.jpg",
    "https://ultralytics.com/images/zidane.jpg",
    "https://ultralytics.com/images/bike.jpg"
]
downloaded_images = []
for i, url in enumerate(img_urls):
    img_filename = f"img{i}.jpg"
    # Use requests to download the image
    response = requests.get(url)
    if response.status_code == 200:
        with open(img_filename, 'wb') as f:
            f.write(response.content)
        downloaded_images.append(img_filename)
    else:
        print(f"Failed to download {url}")

# 2) 일괄 예측
if downloaded_images:
    results = model(downloaded_images)
    # 3) 시각화
    for r in results:
        r.show()
else:
    print("No images were successfully downloaded.")

In [ ]:
from google.colab import files

# 1) 이미지 업로드
uploaded = files.upload()
img_path = list(uploaded.keys())[0]

# 2) YOLO 예측
results = model(img_path)
# 3) 결과 시각화
for r in results:
  r.show()

In [ ]:
# 1) 이미지 예측
results = model('sample.jpg')

# 2) 결과를 pandas DataFrame으로 추출
import pandas as pd

for r in results:
    boxes = r.boxes.xyxy.cpu().numpy()
    confs = r.boxes.conf.cpu().numpy()
    labels = r.boxes.cls.cpu().numpy()
    classes = [model.names[int(l)] for l in labels]
    df = pd.DataFrame(boxes, columns=['x1','y1','x2','y2'])
    df['class'] = classes
    df['confidence'] = confs
    print(df)


In [ ]:
# 예: 'person'(사람) 클래스만 시각화 (클래스명은 model.names 참고)
results = model('sample.jpg')
for r in results:
    img = r.orig_img.copy()
    for box, label, conf in zip(r.boxes.xyxy.cpu().numpy(), r.boxes.cls.cpu().numpy(), r.boxes.conf.cpu().numpy()):
        if model.names[int(label)] == 'person':
            x1, y1, x2, y2 = map(int, box)
            cv2.rectangle(img, (x1, y1), (x2, y2), (255,0,0), 2)
            cv2.putText(img, f'person {conf:.2f}', (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title('Only Person Detection')
    plt.show()
